In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display
from scipy.stats import mannwhitneyu, wilcoxon

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.utils.qt_helper import gui_fnames, gui_fpath
from mesostat.metric.metric import MetricCalculator
from mesostat.utils.hdf5_io import DataStorage

from lib.sych.data_fc_db_raw import DataFCDatabase
import lib.analysis.bulk_metrics as bulk_metrics

%load_ext autoreload
%autoreload 2

In [ ]:
# tmp_path = root_path_data if 'root_path_data' in locals() else "./"
params = {}
# params['root_path_data'] = './'
params['root_path_data'] = '/media/alyosha/Data/TE_data/yarodata/sych_preprocessed'
#params['root_path_data'] = gui_fpath('h5path', './')

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
ds = DataStorage('sych_result_bulk_metrics.h5')

In [ ]:
mc = MetricCalculator(serial=True, verbose=False)

# 1. Bulk Metrics vs Time

* Avg over mice
* All-region-all-types
* Expert vs Naive
* Go/NOGO/MISS/FA

# Mean

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, 'mean', 's', 'time', verbose=False,
                               trialTypeNames='auto', perfNames='auto')

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'mean', 'time', verbose=False)

# Variance

**TODO**
* Plot trial variance relative to temporal variance
* Plot mean with variance together

### Pros/Cons of Baseline Normalization
* DFF-Trial
    - Pos: Removes dynamic baseline changing on the order of trials.
    - Pos: Under assumption of signal-free pre-trial interval, baseline removal enhances relative change in significant activity during trial.
    - Neg: In presence of correlation between pre-trial interval and trial signals, this procedure destroys information during trial.

* DFF-Session vs ZScore-Session
    - Both linear transforms
    - Mean is more meaningful for DFF if pre-trial interval is at least somewhat stable
    - Va

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, 'varmean', 's', 'time', verbose=False,
                               trialTypeNames='auto', perfNames='auto')

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'varmean', 'time', ylim=[0,None], verbose=False)

### Test variance across channels for interesting interval

* Average signal over texture presentation interval
* Compute variance over trials for each channel
* Compare channels

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, 'varmean', 'p', 'channel', cropTime=('TEX', (3, 3.5)), verbose=False)

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'varmean', 'channel', yscale='log', verbose=False, # ylim=[0.005,2],
                              dropCols=['cropTime'])

# Effective Rank

### ByTime

In [ ]:
bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "rank_effective", 's', 'time', verbose=False,
                               trialTypeNames='auto', perfNames='auto')

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'rank_effective', 'time', ylim=[1, None], verbose=False) # ylim=[1,48]

### BySession

In [ ]:
intervDict = {
    'TEX' : (3.0, 3.5),
    'REW' : (6.0, 6.5)
}

for intervName, interv in intervDict.items():
    bulk_metrics.metric_mouse_bulk_vs_session(dataDB, mc, ds, "rank_effective", 'session',
                                              perfNames='auto', verbose=False, cropTime=(intervName, interv))

In [ ]:
bulk_metrics.plot_metric_bulk(ds, 'rank_effective', 'session', ylim=[1,None], verbose=False)

# Total Correlation

In [ ]:
# bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "avg_entropy", 'sp', 'time-channel', verbose=False)
# bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "avg_entropy", 's', 'time', verbose=False)

bulk_metrics.metric_mouse_bulk(dataDB, mc, ds, "avg_TC", 's', 'time', verbose=False)

In [ ]:
bulk_metrics.plot_metric_bulk(dataDB, ds, 'avg_TC', 'time', verbose=True) # ylim=[1,48]

In [ ]:
bulk_metrics.plot_TC(dataDB, ds, ylim=None, yscale=None, verbose=True)

### Cleaning up

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(ds.list_dsets_pd().sort_values(by='datetime'))

In [ ]:
ds.delete_by_query(queryDict={"metric" : "rank_effective"}, timestr="2020-11-20 18:00:00")